In [ ]:
# необходимые библиотеки
import pandas as pd
import re

In [ ]:
pd.set_option('display.max_rows', 1300)
pd.set_option('display.max_columns', 50)

In [ ]:
# pd.reset_option('display.max_rows')

In [ ]:
# необходимые данные
igdb =  pd.read_csv('IGDB/igdb.csv', sep = ';')
hltb = pd.read_csv('HLTB/hltb.csv')
mg = pd.read_csv('MG/mg.csv')
mc = pd.read_csv('MC/metacritic.csv', sep = ';')
fin = pd.read_csv('Finfin.csv', sep = ';')

In [ ]:
igdb

In [ ]:
hltb

In [ ]:
mg

In [ ]:
mc

In [ ]:
fin

In [ ]:
# работаем с Metacritic
mc = mc.drop('release_date', axis = 1)

#корректируем случаи некорректного парсинга
mc.loc[4409, 'title'] = 'Else Heart.Break()'
mc.loc[8615, 'title'] = 'Monsters (Probably) Stole My Princess'
# делаем универсальный идентификатор в нижнем регистре
mc.id = mc.title.str.lower().str.replace('&', ' and ').str.replace('#', ' number ').str.replace('vol.', 'vol').str.replace(r'[.]', ' dot ').str.replace('*', ' star ').str.replace('@', ' at ').str.replace(r'[/|]', ' slash ')
mc.id = mc.id.str.strip().str.replace(r"[\[\];,:()!?' \\\\^_-]+", '-').str.replace(r'^-', '').str.replace('-$', '')
mc = mc.drop('title', axis = 1)

# приводим userscore к типу float
mc.user_score = pd.to_numeric(mc.user_score.str.replace(',', '.'), errors = 'coerce')
mc = mc[mc.user_score.notna()]

# приводим платформы к общему виду
# сначала частные случаи некорректного парсинга
mc.loc[4409, 'platform'] = 'PC'
mc.loc[10104, 'platform'] = 'XBOX'
mc.loc[11586, 'platform'] = 'PS2'
mc.loc[14929, 'platform'] = 'GBA'
# потом меняем всё остальное
mc.platform = mc.platform.replace(
    ['Switch', '1999', '2008', '2015', '2013', '2006', '2016', 'Remake', 'Live Arcade', '2004', '2000', '2011', 'Steam Edition', '2003', 'Probably', '2017', '2012', 
     'PS Mobile', '2014', '2001', 'Online Edition', '2005', '2018', '2010', 'All Points Bulletin', '2007', '3DSWare', '2009', 'The War Z'],
    ['SWITCH', 'PC', 'PC', 'PC', 'PC', 'PC', 'PC', 'PC', 'X360', 'PC', 'PC' ,'PC', 'PC', 'PC', 'PSP', 'PC', 'PC', 'VITA', 'PC', 'PC', 'XBOX', 'PC', 'PC', 'PC', 'PC' , 'PC',
    '3DS', 'PC' , 'PC'])

# сортируем колонки для удобства
mc = mc.reset_index(drop = True)
cols = mc.columns.tolist()
cols_new = cols[0:1] + cols[4:5] + cols[3:4] + cols[5:6] + cols[1:3]
mc = mc.reindex(columns = cols_new)

In [ ]:
# работаем с IGDB
# убираем лишние колонки
igdb = igdb.drop(['dev', 'support_dev', 'game_eng', 'themes', 'mult_modes', 'perspective', ], axis = 1)
#убираем игры без издателя, жанра, режимов, даты выпуска
igdb = igdb[(igdb.publisher.notna()) & (igdb.publisher != 'Try Again') & (igdb.release_dates.notna()) & (igdb.genres.notna()) & (igdb.game_modes.notna())]

# делаем универсальный идентификатор в нижнем регистре
igdb.id = igdb.title
igdb = igdb.drop('title', axis = 1)

# разбиваем даты
igdb.release_dates = igdb.release_dates.str.split('; ')
igdb = igdb.explode('release_dates')
igdb[['release_date', 'platform']] = igdb.release_dates.str.split(' - ', expand = True)
igdb = igdb.drop('release_dates', axis = 1)
# приводим дату к формату datetime
igdb.release_date = pd.to_datetime(igdb.release_date, errors = 'coerce')
igdb = igdb[igdb.release_date.notna()]
igdb['year'] = igdb.release_date.dt.year

# приводим платформы к общему виду
igdb.platform = igdb.platform.mask((igdb.platform == "PlayStation Network") & (igdb.release_date >= '2013-11-15'), 'PS4')
igdb.platform = igdb.platform.mask((igdb.platform == "Xbox Live Arcade") & (igdb.release_date >= '2013-11-22'), 'XONE')
igdb.platform = igdb.platform.mask((igdb.platform == "Virtual Console (Nintendo)") & (igdb.release_date >= '2012-11-18'), 'WIIU')
igdb.platform = igdb.platform.replace(
    ['PC (Microsoft Windows)', 'Xbox One', 'Wii U', 'PlayStation 4', 'Nintendo Switch', 'PlayStation 3', 'Xbox 360', 'PlayStation 2', 'Wii', 'Nintendo DS', 
     'PlayStation', 'Nintendo 64', 'PlayStation Portable', 'Nintendo 3DS', 'PlayStation Vita', 'Nintendo GameCube', 'Xbox', 'Dreamcast', 'Nintendo DSi', 
     'Game Boy Advance', 'PlayStation VR', 'SteamVR', 'Oculus VR', 'Windows Mixed Reality', 'New Nintendo 3DS', 'PlayStation Network', 'Xbox Live Arcade',
    'WiiWare', 'Virtual Console (Nintendo)', 'Nintendo eShop'], 
    ['PC', 'XONE', 'WIIU', 'PS4', 'SWITCH', 'PS3', 'X360', 'PS2', 'WII', 'DS', 'PS', 'N64', 'PSP', '3DS', 'VITA', 'GC', 'XBOX', 'DC', 'DS', 'GBA', 'PS4', 'PC', 
     'PC', 'PC', '3DS', 'PS3', 'X360', 'WII', 'WII', '3DS'])
# убираем образовавшиеся дубликаты
igdb = igdb[igdb.duplicated() == False]

# разбиваем издателей - останутся слова вроде Inc., LLC, Co., но при анализе их можно просто не учитывать
igdb.publisher = igdb.publisher.str.split(', ')
igdb = igdb.explode('publisher')

# кодируем жанры
temp = igdb.genres.str.get_dummies(', ')
igdb[temp.columns] = temp

igdb['action'] = igdb[:][['Arcade', 'Fighting', "Hack and slash/Beat 'em up", 'Music', 'Pinball', 'Platform']].sum(axis =1) > 0
igdb.action = igdb.action.mask(igdb.action, 1).where(igdb.action, 0)
igdb['strategy'] = igdb[:][['Real Time Strategy (RTS)', 'Strategy', 'Tactical', 'Turn-based strategy (TBS)']].sum(axis =1) > 0
igdb.strategy = igdb.strategy.mask(igdb.strategy, 1).where(igdb.strategy, 0)
igdb['shooter'] = igdb.Shooter
igdb['sports'] = igdb.Sport
igdb['adventure'] = igdb[:][['Adventure', 'Point-and-click', 'Visual Novel']].sum(axis =1) > 0
igdb.adventure = igdb.adventure.mask(igdb.adventure, 1).where(igdb.adventure, 0)
igdb['rpg'] = igdb['Role-playing (RPG)']
igdb['simulation'] = igdb['Simulator']
igdb['puzzle'] = igdb['Puzzle']
igdb['vehicle'] = igdb['Racing']
igdb['trivia'] = igdb['Quiz/Trivia']
igdb = igdb.drop(
    ['Adventure', 'Arcade', 'Fighting', "Hack and slash/Beat 'em up", 'Indie', 'Music', 'Pinball', 'Platform', 'Point-and-click', 'Puzzle', 'Quiz/Trivia',
     'Racing', 'Real Time Strategy (RTS)', 'Role-playing (RPG)', 'Shooter', 'Simulator', 'Sport', 'Strategy', 'Tactical', 'Turn-based strategy (TBS)',
     'Visual Novel'], axis = 1)
igdb = igdb[igdb.loc[:, 'action':'trivia'].sum(axis = 1) > 0]

# кодируем режимы
temp = igdb.game_modes.str.get_dummies(', ')
igdb[temp.columns] = temp
igdb['mp'] = igdb.Multiplayer.mask(igdb.Multiplayer == 0, igdb['Battle Royale'])
igdb['mp'] = igdb.mp.mask(igdb.mp == 0, igdb['Co-operative'])
igdb['mp'] = igdb.mp.mask(igdb.mp == 0, igdb['Massively Multiplayer Online (MMO)'])
igdb['mp'] = igdb.mp.mask(igdb.mp == 0, igdb['Split screen'])
igdb['sp'] = igdb['Single player']
igdb = igdb.drop(['Single player', 'Split screen', 'Co-operative', 'Multiplayer', 'Battle Royale', 'Massively Multiplayer Online (MMO)'], axis = 1)

#кодируем франчайз
igdb.series = igdb.series.where(igdb.series.isna(), 1).fillna(0)

# выкидываем лишнее и сортируем колонки для удобства
igdb = igdb[igdb.duplicated() == False]
igdb = igdb.drop(['game_modes', 'genres', 'release_date'], axis = 1)
cols = igdb.columns.tolist()
cols_new = cols[0:1] + cols[5:6] + cols[4:5] + cols[6:7] + cols[1:4] + cols[7:]
igdb = igdb.reindex(columns = cols_new)
igdb = igdb.reset_index(drop=True)

In [ ]:
# работаем с HowLongToBeat
# убираем лишние колонки
hltb = hltb.drop(['type', 'parent', 'sp_pooled', 'sp_average', 'coop_pooled', 'coop_average', 'vs_pooled', 'vs_average', 'date_na', 'date_eu', 'date_jp'], axis = 1)
#убираем игры без платформы или длительности
hltb = hltb[
    (hltb.platform.notna()) &  
    ((hltb.sp_median.notna()) | (hltb.coop_median.notna()) | (hltb.vs_median.notna()))]

# делаем универсальный идентификатор в нижнем регистре
hltb['id'] = hltb.title.str.lower().str.replace('&', ' and ').str.replace('#', ' number ').str.replace('vol.', 'vol').str.replace(r'[.]', ' dot ').str.replace('*', ' star ').str.replace('@', ' at ').str.replace(r'[/|]', ' slash ')
hltb.id = hltb.id.str.strip().str.replace(r"[\[\];,:()!?' \\\\^_-]+", '-').str.replace(r'^-', '').str.replace('-$', '')
hltb = hltb.drop('title', axis = 1)

# разбиваем платформы
hltb.loc[:, 'platform'] = hltb.platform.str.split(', ')
hltb = hltb.explode('platform')
hltb.loc[:, 'platform'] = hltb.platform.replace(
    ['Xbox 360', 'Xbox One', 'PlayStation 2', 'PlayStation 4', 'PlayStation 3', 'Wii U', 'PlayStation', 'Nintendo 3DS', 'Nintendo DS', 'Dreamcast', 'Nintendo 64',
     'Wii', 'Nintendo GameCube', 'Arcade,', 'Xbox', 'PlayStation Vita', 'Nintendo Switch', 'Game Boy Advance', 'Oculus Rift', 'PlayStation VR', 'HTC Vive',
    'Oculus Quest'], 
    ['X360', 'XONE', 'PS2', 'PS4', 'PS3', 'WIIU', 'PS', '3DS', 'DS', 'DC', 'N64', 'WII', 'GC', 'Arcade', 'XBOX', 'VITA', 'SWITCH', 'GBA', 'PC',
    'PS4', 'PC', 'PC'])
hltb = hltb[hltb.duplicated() == False]

# приводим игровое время к удобному формату
hltb.loc[:,'sp_median'] = hltb.sp_median.str.extract(r"(\d+)(?=h)", expand = False).fillna(0).astype(int)*60 + hltb.sp_median.str.extract(r"(\d+)(?=m)", expand = False).fillna(0).astype(int)
hltb.loc[:,'coop_median'] = hltb.coop_median.str.extract(r"(\d+)(?=h)", expand = False).fillna(0).astype(int)*60 + hltb.coop_median.str.extract(r"(\d+)(?=m)", expand = False).fillna(0).astype(int)
hltb.loc[:, 'vs_median'] = hltb.vs_median.str.extract(r"(\d+)(?=h)", expand = False).fillna(0).astype(int)*60 + hltb.vs_median.str.extract(r"(\d+)(?=m)", expand = False).fillna(0).astype(int)
hltb.loc[:, 'mp_median'] = hltb.vs_median + hltb.coop_median

# выкидываем лишнее и сортируем колонки для удобства
hltb = hltb.drop(['coop_median', 'vs_median'], axis = 1)
hltb = hltb.reindex(columns = ['id', 'platform', 'sp_median', 'mp_median'])
hltb = hltb.reset_index(drop = True)

In [ ]:
# работаем с MobyGames
# убираем лишние колонки
mg = mg.drop(['release', 'n_thanks'], axis = 1)

# делаем универсальный идентификатор в нижнем регистре
mg['id'] = mg.title.str.lower().str.replace('&', ' and ').str.replace('#', ' number ').str.replace('vol.', 'vol').str.replace(r'[.]', ' dot ').str.replace('*', ' star ').str.replace('@', ' at ').str.replace(r'[/|]', ' slash ')
mg.id = mg.id.str.strip().str.replace(r"[\[\];,:()!?' \\\\^_-]+", '-').str.replace(r'^-', '').str.replace('-$', '')
mg = mg.drop('title', axis = 1)

# разбиваем платформы
mg.loc[:, 'platform'] = mg.platform.str.split(', ')
mg = mg.explode('platform')
mg.loc[:, 'platform'] = mg.platform.replace(
    ['Windows', 'GameCube', 'PlayStation 2', 'Xbox', 'PlayStation 3', 'Xbox 360', 'Game Boy Advance', 'Wii U', 'Wii', 'PlayStation', 'Nintendo 64', 'Xbox One', 
     'Windows 3.x', 'PS Vita', 'Nintendo DS', 'PlayStation 4', 'Nintendo Switch', 'Windows Apps', 'Nintendo 3DS', 'Nintendo DSi', 'Dreamcast', 
     'New Nintendo 3DS', 'Oculus Quest'],
    ['PC', 'GC', 'PS2', 'XBOX', 'PS3', 'X360', 'GBA', 'WIIU', 'WII', 'PS', 'N64', 'XONE', 'PC', 'VITA', 'DS', 'PS4', 'SWITCH', 'PC', '3DS', 'DS', 'DC', '3DS', 'PC'])
mg = mg[mg.duplicated() == False]

# достаём из строк число разработчиков
mg.loc[:, 'n_devs'] = mg.n_devs.str.extract(r'\((\d+) people\)', expand=False)
mg.loc[:, 'n_staff'] = mg.n_staff.str.extract(r'(\d+)', expand=False).fillna(mg.n_devs)
mg = mg[mg.n_staff.notna()]
mg.loc[:, 'n_devs'] = mg.n_staff.astype(int)
mg = mg.drop('n_staff', axis = 1)

# сортируем колонки для удобства
mg = mg.reindex(columns = ['id', 'platform', 'n_devs'])

In [ ]:
# работаем с финансовым датасетом
fin.loc[:,'stock_var'] = fin.stock_var.str.replace(',', '.').astype(float)
fin['sust'] = fin.op/(fin.stock_var)**0.5
fin = fin.reindex(columns = ['publisher', 'year', 'yaf', 'op', 'stock_var'])

In [ ]:
# корректируем названия издателей для корректного соединения с 
igdb.publisher = igdb.publisher.replace(['Deep Silver', '505 Games', 'Rockstar Games', 'EA Sports', 'NIS America', 'THQ Nordic', '2K Games',
                                         'Rising Star Games', 'Compile Heart', 'Atlus USA', 'Ghostlight Ltd.', 'Namco Bandai Games', 'Bandai Namco Games', 
                                         'Koei Tecmo', '2K Sports', 'Nordic Games Publishing', 'Square', 'Ubisoft', 'Spike ChunSoft', 'JoWooD Entertainment AG',
                                        'Square Electronic Arts', 'Spike', 'EA Sports BIG', 'DreamCatcher Interactive', 'Tecmo Co.', 'MacSoft Games', 'Natsume',
                                        'Koei Tecmo America Corporation', 'Nintendo of Europe', 'Koei Tecmo Europe', 'Nintendo of America', 'Nintendo Australia',
                                        'Bandai Namco Entertainment Europe', 'Bandai Namco Entertainment America', 'Marvelous Entertainment', 'EA Games',
                                        'Marvelous USA', 'Square Soft', 'JoWooD Productions Software', 'The Adventure Company', 'THQ Wireless', 
                                        'Bandai Namco Holdings', 'DreamCatcher Games', 'Enix Corporation', 'Koch Media', 'Konami Digital Entertainment',
                                        'Square Enix Collective', 'Ubisoft Europe', '505 Game Street', 'Electronic Arts Victor', 'Interplay', 'Marvelous AQL',
                                        'Idea Factory International', 'NIS Europe', 'Kadokawa Games', 'Namco Bandai Partners', 'Marvelous Interactive', 'Activision Value',
                                        'Activision Publishing', 'Marvelous AQL Europe', 'Sierra Studios', 'Sierra Online', 'Sierra', 'EA Play', 'Capcom Entertainment',
                                        'Private Division', 'Play THQ', 'THQ Asia Pacific', 'Ubisoft EMEA', 'Ubisoft Montreal', 'Victor Interactive Software', 'Spike Chunsoft',
                                        'EA Mobile', 'EA Originals', 'EA Partners', 'Curve Studios', 'D3Publisher of America', 'Eidos Interactive Europe',
                                         'Enix America Corporation', 'Activision Publishing Minneapolis Inc.', 'Actvision', 'Capcom Europe', 'Capcom USA',
                                        'NAMCO BANDAI Games America Inc.', 'Namco Bandai Europe', 'Namco Limited', 'NatsumeAtari Inc.', 'Infogrames North America',
                                        'Infrogrames'], 
                       ['Embracer Group', 'Digital Bros', 'Take-Two Interactive', 'Electronic Arts', 'Nippon Ichi Software', 'Embracer Group', 'Take-Two Interactive',
                        'Marvelous', 'Idea Factory', 'Atlus', 'Majesty House Group', 'Bandai Namco Entertainment', 'Bandai Namco Entertainment',
                        'Koei Tecmo Games', 'Take-Two Interactive', 'Embracer Group', 'Square Enix', 'Ubisoft Entertainment', 'Kadokawa', 'Embracer Group',
                        'Square Enix', 'Kadokawa', 'Electronic Arts', 'Embracer Group', 'Koei Tecmo Games', 'Bandai Namco Entertainment', 
                        'Bandai Namco Entertainment', 'Koei Tecmo Games', 'Nintendo', 'Koei Tecmo Games', 'Nintendo', 'Nintendo', 'Bandai Namco Entertainment',
                        'Bandai Namco Entertainment', 'Marvelous', 'Electronic Arts', 'Marvelous', 'Square Enix', 'Embracer Group', 'Embracer Group', 'THQ',
                        'Bandai Namco Entertainment', 'Embracer Group', 'Square Enix', 'Embracer Group', 'Konami', 'Square Enix', 'Ubisoft Entertainment',
                        'Digital Bros', 'Electronic Arts', 'Interplay Entertainment', 'Marvelous', 'Idea Factory', 'Nippon Ichi Software', 'Kadokawa',
                        'Bandai Namco Entertainment', 'Marvelous', 'Activision', 'Activision', 'Marvelous', 'Sierra Entertainment', 'Sierra Entertainment', 
                        'Sierra Entertainment', 'Electronic Arts', 'Capcom', 'Take-Two Interactive', 'THQ', 'THQ', 'Ubisoft Entertainment', 'Ubisoft Entertainment',
                       'Marvelous', 'Kadokawa', 'Electronic Arts', 'Electronic Arts', 'Electronic Arts', 'Curve Digital', 'D3 Publisher', 'Eidos Interactive', 'Square Enix',
                       'Activision', 'Activision', 'Capcom', 'Capcom', 'Bandai Namco Entertainment', 'Bandai Namco Entertainment', 'Bandai Namco Entertainment',
                       'Bandai Namco Entertainment', 'Atari', 'Atari'])

igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Atlus') & (igdb.year >= 2010), 'Index Corporation')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Index Corporation') & (igdb.year >= 2013), 'Sega')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Activision') & (igdb.year >= 2008), 'Activision Blizzard')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Bandai Namco Entertainment') & (igdb.year < 2005), 'Bandai')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'THQ') & (igdb.year > 2012), 'Embracer Group')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Atari') & (igdb.year <= 2001), 'Hasbro')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'XSEED Games') & (igdb.year >= 2011), 'Marvelous')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Eidos Interactive') & (igdb.year >= 2009), 'Square Enix')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Blizzard Entertainment') & (igdb.year >= 2008), 'Activision Blizzard')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Koei') & (igdb.year >= 2010), 'Koei Tecmo Games')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'D3 Publisher') & (igdb.year >= 2009), 'Bandai Namco Entertainment')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Namco') & (igdb.year >= 2005), 'Bandai Namco Entertainment')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'FromSoftware') & (igdb.year >= 2014), 'Kadokawa')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Sierra Entertainment') & (igdb.year >= 2008), 'Activision Blizzard')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Koei Tecmo Games') & (igdb.year < 2010), 'Koei')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Curve Digital') & (igdb.year > 2016), 'Catalis Group')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Catalis Group') & (igdb.year > 2016), 'North Edge Capital')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Bandai') & (igdb.year >= 2005), 'Bandai Namco Entertainment')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Infogrames') & (igdb.year >= 2008), 'Atari')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'Infogrames Entertainment') & (igdb.year >= 2008), 'Atari')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'PopCap Games') & (igdb.year >= 2011), 'Electronic Arts')
igdb.publisher = igdb.publisher.mask((igdb.publisher == 'AQ Interactive') & (igdb.year >= 2011), 'Marvelous')

In [ ]:
# соединяем датасеты
igdb = pd.merge(fin, igdb, how = 'inner', on = ['publisher', 'year'])
main = pd.merge(pd.merge(pd.merge(mg, hltb, how = 'inner', on = ['id', 'platform']), igdb, how = 'inner', on=['id', 'platform']), mc, how='inner', on=['id', 'platform'])
main = main[main.duplicated() == False]
main['sust'] = main.op/main.stock_var
main = main.reindex(columns = [
    'id', 'platform', 'year', 'publisher', 'yaf', 'series', 
    'n_devs', 
    'sp', 'sp_median', 'mp', 'mp_median', 
    'action', 'shooter', 'adventure', 'rpg', 'strategy', 'simulation', 'sports', 'vehicle', 'puzzle', 'trivia', 
    'crit_score', 'user_score',
    'op', 'stock_var', 'sust'])

In [ ]:
main

In [ ]:
# main.to_csv('data.csv', index = False)

In [ ]:
main_temp = pd.concat([main, main.platform.str.get_dummies()], axis = 1)
main_temp['n_games'] = main_temp.id
main_temp = main_temp.reindex(columns = [
    'n_games', 'year', 'publisher', 'yaf', 'series', 
    'n_devs', 
    'sp', 'sp_median', 'mp', 'mp_median', 
    'action', 'shooter', 'adventure', 'rpg', 'strategy', 'simulation', 'sports', 'vehicle', 'puzzle', 'trivia',
    'PC', 'PS4', 'XONE', 'SWITCH', 'WIIU', '3DS', 'VITA', 'WII', 'X360', 'PS3', 'PSP', 'DS', 'PS2', 'XBOX', 'GC', 'GBA', 'PS', 'N64', 'DC',
    'crit_score', 'user_score',
    'op', 'stock_var', 'sust'])

In [ ]:
main_temp

In [ ]:
main_reg = main_temp.groupby(['year', 'publisher']).agg(
    {
        'n_games': 'count',
        'yaf': 'mean',
        'series': 'sum',
        'n_devs': 'median',
        'sp': 'sum',
        'sp_median': 'median',
        'mp': 'sum',
        'mp_median': 'median',
        'action': 'sum',
        'strategy': 'sum',
        'shooter': 'sum',
        'sports': 'sum',
        'adventure': 'sum',
        'rpg': 'sum',
        'simulation': 'sum',
        'puzzle': 'sum',
        'vehicle': 'sum',
        'trivia': 'sum',
        'PC': 'sum',
        'PS4': 'sum',
        'XONE': 'sum',
        'SWITCH': 'sum',
        'WIIU': 'sum',
        'DS': 'sum',
        'VITA': 'sum',
        'WII': 'sum',
        'X360': 'sum',
        'PS3': 'sum',
        '3DS': 'sum',
        'PSP': 'sum',
        'DS': 'sum',
        'PS2': 'sum',
        'XBOX': 'sum',
        'GC': 'sum',
        'GBA': 'sum',
        'PS': 'sum',
        'N64': 'sum',
        'DC': 'sum',
        'crit_score': 'median',
        'user_score': 'median',
        'op': 'mean',
        'stock_var': 'mean',
        'sust': 'mean'
    })

main_reg.loc[:, 'op'] = main_reg.op/1000000
main_reg.loc[:, 'sust'] = main_reg.sust/1000000

main_reg['unique_genres'] = main_reg.loc[:, 'action':'trivia'].mask(main_reg.loc[:, 'action':'trivia'] > 0, 1).sum(axis = 1)
main_reg['unique_platforms'] = main_reg.loc[:, 'PC':'DC'].mask(main_reg.loc[:, 'PC':'DC'] > 0, 1).sum(axis = 1)

main_reg.loc[:, 'action':'trivia'] = main_reg.loc[:, 'action':'trivia'].div(main_reg.loc[:,'n_games'], axis = 0).multiply(100)
main_reg.loc[:, 'sp'] = main_reg.loc[:, 'sp'].div(main_reg.loc[:,'n_games'], axis = 0).multiply(100)
main_reg.loc[:, 'mp'] = main_reg.loc[:, 'mp'].div(main_reg.loc[:,'n_games'], axis = 0).multiply(100)
main_reg.loc[:, 'series'] = main_reg.loc[:, 'series'].div(main_reg.loc[:,'n_games'], axis = 0).multiply(100)

main_reg.loc[:, 'DC'] = main_reg.loc[:, 'DC'].div(main_reg.loc[:,'n_games'], axis = 0).multiply(100)
main_reg['P_sony'] = main_reg.loc[:, ['PS4', 'VITA', 'PS3', 'PSP', 'PS2', 'PS']].sum(axis = 1).div(main_reg.loc[:,'n_games'], axis = 0).multiply(100)
main_reg['P_xbox'] = main_reg.loc[:, ['XONE', 'X360', 'XBOX']].sum(axis = 1).div(main_reg.loc[:,'n_games'], axis = 0).multiply(100)
main_reg['P_nintendo'] = main_reg.loc[:, ['SWITCH', 'WIIU', 'WII', 'GC', '3DS', 'DS', 'N64', 'GBA']].sum(axis = 1).div(main_reg.loc[:,'n_games'], axis = 0).multiply(100)
main_reg['station'] = main_reg.loc[:, ['PC', 'SWITCH', 'WIIU', 'WII', 'GC', 'PS4', 'PS3', 'PS2', 'PS', 'XBOX', 'XONE', 'X360', 'N64', 'DC']].sum(axis = 1).div(main_reg.loc[:,'n_games'], axis = 0).multiply(100)
main_reg['handheld'] = main_reg.loc[:, ['SWITCH', 'VITA', 'PSP', '3DS', 'DS', 'GBA']].sum(axis = 1).div(main_reg.loc[:,'n_games'], axis = 0).multiply(100)

main_reg = main_reg.reindex(columns = [
    'yaf', 
    'series',
    'n_devs',
    'sp', 
    'sp_median', 
    'mp',
    'mp_median',
    'action', 'shooter', 'adventure', 'rpg', 'strategy', 'simulation', 'sports', 'vehicle', 'puzzle', 'trivia',
    'unique_genres',
    'P_sony', 'P_xbox', 'P_nintendo', 
    'DC',
   'handheld', 'station',
    'unique_platforms',
    'crit_score', 
    'user_score',
    'op', 'stock_var', 'sust'])

In [ ]:
main_reg

In [ ]:
# main_reg.to_csv('data_reg.csv')